# Explanatory Data Analysis
Перед нами стоит задача провести EDA. В первую очередь мы попробуем прочитать предложенный файл и посмотреть на результат. Но перед эти импортируем необходимые для этого библиотеки.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
%matplotlib inline
import plotly as py
import cufflinks as cf
from plotly.offline import iplot
py.offline.init_notebook_mode(connected=True)
cf.go_offline()

Прочитаем полученный файл

In [2]:
info = pd.read_csv('D:/Проекты/Work/texts.csv')
info.head()

,id,txt,gender,birthdate,create_date,source,additional_info
0,3181,Здравствуйте Евгений Альбертович! В октябре 20...,NaN,NaN,2021-04-10,vladmedicina.ru/faq/answers/73190.htm,NaN
1,3182,Здравствуйте. Как на этом сайте оставить отзыв?,NaN,NaN,2021-04-06,vladmedicina.ru/faq/answers/73172.htm,NaN
2,3183,"Добрый день, Евгений Альбертович! Ребенку 3,5 ...",female,NaN,2021-04-02,vladmedicina.ru/faq/answers/73154.htm,NaN
3,3184,"Здравствуйте, Алексей Леонтьевич! Скажите, пос...",NaN,NaN,2020-12-08,vladmedicina.ru/faq/answers/72437.htm,NaN
4,3185,Здравствуйте! Хочу оставить отзыв. Восемь лет ...,female,NaN,2021-03-31,vladmedicina.ru/faq/answers/73141.htm,NaN


Далее, для упрощения анализа оставим только необходимые для этого столбцы. Для этого воспользуемся методом drop().

In [3]:
info.drop(labels=['id', 'birthdate', 'additional_info', 'source'], axis=1, inplace=True)

Давайте посмотрим, есть ли в нашем наборе данных какие-либо нулевые значения:

In [4]:
info.isnull().sum()

txt               0
gender         5075
create_date       0
dtype: int64

Видно, что в столбце gender присутствуют пустые значения. Для того, чтобы было проще проводить анализ предложенных данных, воспользуемся методом dropna() для удаления нулевых значений.

In [5]:
info.dropna(subset=['gender'], inplace=True)

Мы можем наглядно рассмотреть всевозможные сокращения, которые используются в русском языке и в последствии создадим словарь с такими сокращениями, дабы текст был более приятным на вид и понятным.

In [6]:
contraction = {"д/сад": "детский сад", "г.": "город", "мм": "миллиметр", "т.д": "так далее", "см": "сантиметр",
               "таб/день": "таблетки в день", "кг": "киллограмм", "уд/мин": "ударов в минуту"}
def cont_to_exp(x):
    if type(x) is str:
        for key in contraction:
            value = contraction[key]
            x = x.replace(key, value)
        return x
    else:
        return x
info['txt'] = info['txt'].apply(lambda x: cont_to_exp(x))

# Очистка текстовых данных в Python
Как правило, текстовые данные содержат много шума либо в виде символов, либо в виде знаков препинания и стоп-слов. Следовательно, возникает необходимость очистить текст не только для того, чтобы сделать его более понятным, но и для лучшего понимания.

Мы можем видеть, что у нас в тексте есть нежелательные сокращения по типу 3.5 года, то есть цифры, которые можно записать в виде текста, а также знаки препинания.

# Нижний регистр
В НЛП модели трактуют такие слова, как график(план работы) и график(художник), по-разному, даже если они одинаковы. Поэтому, чтобы решить эту проблему, мы пишем слова в нижнем регистре. Здесь я использую функцию lower (), доступную в Python, для преобразования текста в нижний регистр:

In [7]:
info['cleaned']=info['txt'].apply(lambda x: x.lower())

# Удаление цифр и слов, содержащих цифры
Далее нам нужно удалить числа и слова, содержащие цифры. Я делаю это потому, что цифры и слова, содержащие цифры, не придают большого значения основным словам. Для этого я использую регулярные выражения с лямбда-функциями.

In [8]:
info['cleaned']=info['cleaned'].apply(lambda x: re.sub('\w*\d\w*','', x))

# Удаление пунктуации
Пунктуация - это знаки языка, такие как запятые, дефисы, точки и т.д. Они важны для грамматики языка, но не для анализа текста. Поэтому их нужно удалить:

In [9]:
info['cleaned']=info['cleaned'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))
info['cleaned']=info['cleaned'].apply(lambda x: re.sub(' +',' ',x))
info['cleaned']=info['cleaned'].apply(lambda x: re.sub("^\s+|\n|\r|\s+$", ' ', x))

# Подготовка текстовых данных для исследовательского анализа данных (EDA)
Мы уже очистили наши данные и подготовили корпус. В этом разделе мы создадим матрицу терминов документа, которую позже будем использовать в нашем анализе.

В этом разделе сделаем следующее:

Удаление стоп-слов

Лемматизация

# Стоп-слова
Стоп-слова - это наиболее распространенные слова в таких языках, как «я», «это», «есть», «в», которые не придают большого значения значению документа. Эти значения будут удалены, чтобы уменьшить размер набора данных и сосредоточить внимание на значимых словах.

# Лемматизация
Лемматизация - это систематический процесс сведения токена к его лемме. Он использует словарь, структуру слова, часть речевых тегов и грамматические отношения для преобразования слова в его базовую форму.

Для этого я буду использовать SpaCy для удаления игнорируемых слов и лемматизации.

In [10]:
import spacy
from spacy.lang.ru import Russian
nlp = Russian()
nlp = spacy.load('ru_core_news_sm',disable=['parser', 'ner'])
info['lemmatized']=info['cleaned'].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if (token.is_stop==False)]))
info['lemmatized'].head()
info_grouped=info[['txt','lemmatized']].groupby(by='txt').agg(lambda x:' '.join(x))

# Посчитаем базовые статистики по текстам

In [11]:
from textblob import TextBlob
info['review_len'] = info['lemmatized'].apply(lambda x: len(x))  # Считаем количество символов
info['w_count'] = info['lemmatized'].apply(lambda x: len(x.split()))  # Общее количество слов в данном наборе


# Средняя длина слова
def middle_words(x):
    words = x.split()
    word_len = 0
    for word in words:
        word_len = word_len + len(word)
    return word_len/len(words)


info['middle_words'] = info['lemmatized'].apply(lambda x: middle_words(x))  # Поиск средней длины слова
info.head()

,txt,gender,create_date,cleaned,lemmatized,review_len,w_count,middle_words
2,"Добрый день, Евгений Альбертович! Ребенку 3,5 ...",female,2021-04-02,добрый день евгений альбертович ребенку года с...,добрый день евгений альбертович ребёнок год ст...,332,46,6.239130
4,Здравствуйте! Хочу оставить отзыв. Восемь лет ...,female,2021-03-31,здравствуйте хочу оставить отзыв восемь лет мо...,здравствовать хотеть оставить отзыв восемь год...,530,59,8.000000
5,Здравствуйте! В 2018 году ребенку (сейчас ему ...,female,2021-03-29,здравствуйте в году ребенку сейчас ему лет уда...,здравствовать год ребёнок сейчас год удалить а...,274,34,6.852941
8,"Здравствуйте! Скажите, пожалуйста, может ли ми...",female,2021-02-24,здравствуйте скажите пожалуйста может ли минол...,здравствовать сказать пожалуйста ли минолексин...,122,15,7.200000
12,"Здравствуйте, Евгений Альбертович! Куда и с ка...",female,2021-02-14,здравствуйте евгений альбертович куда и с каки...,здравствовать евгений альбертович куда какой о...,157,20,6.900000


# Визуализация
Визуализируем полученные результаты. Для начала построим график к количеству символов

In [12]:
info['review_len'].iplot(kind='hist', xTitle = "Количество символов", yTitle = "Количество отзывов, имеющих столько символов", title = "Количество символов")

Теперь для количества слов

In [13]:
info['w_count'].iplot(kind='hist', xTitle = "Количество слов", yTitle = "Количество отзывов, имеющих столько слов", title = "Количество слов")


Ну и в итоге для средней длины слова

In [14]:
info['middle_words'].iplot(kind='hist', xTitle = "Средняя длина слов", yTitle = "Количество отзывов, имеющих такую среднюю длину слова", title = "Средняя длина слов")

# Классификация текстов
Вот мы и подошли к моменту, когда нам нужно проклассифицировать тексты пользователей. Для этого я создал файл Excel, где создал таблицу с двумя столбцами (Text, Topic). В столбце Topic указывается тип сообщения (Жалобы, Запись на прием), а в Text содержится текст подобных сообщений.

In [16]:
# Assign spreadsheet filename to `file`
file = 'D:/Проекты/New popitka/Книга1.xlsx'

# Load spreadsheet
xl = pd.ExcelFile(file)

# Load a sheet into a DataFrame by name: df1
df1 = xl.parse()

In [17]:
def preobrazovanie(z):
    z = z.apply(lambda x: x.lower())
    z = z.apply(lambda x: re.sub('\w*\d\w*','', x))
    z = z.apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))
    z = z.apply(lambda x: re.sub(' +',' ',x))
    z = z.apply(lambda x: re.sub("^\s+|\n|\r|\s+$", ' ', x))
    return z

In [19]:
txt = preobrazovanie(df1['Text'])

Подключим необходимые библиотеки

In [21]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

После чего обучим нашу модель

In [23]:
X = df1['Text']
Y = df1['Topic']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
logreg = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', LogisticRegression(n_jobs=1,C=1e5))])

In [24]:
%%time
logreg.fit(X_train, Y_train)

Wall time: 27.9 ms


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(C=100000.0, n_jobs=1))])

In [26]:
info['tech_pred'] = logreg.predict(info['cleaned'])

In [27]:
info

,txt,gender,create_date,cleaned,lemmatized,review_len,w_count,middle_words,tech_pred
2,"Добрый день, Евгений Альбертович! Ребенку 3,5 ...",female,2021-04-02,добрый день евгений альбертович ребенку года с...,добрый день евгений альбертович ребёнок год ст...,332,46,6.239130,Жалобы
4,Здравствуйте! Хочу оставить отзыв. Восемь лет ...,female,2021-03-31,здравствуйте хочу оставить отзыв восемь лет мо...,здравствовать хотеть оставить отзыв восемь год...,530,59,8.000000,Жалобы
5,Здравствуйте! В 2018 году ребенку (сейчас ему ...,female,2021-03-29,здравствуйте в году ребенку сейчас ему лет уда...,здравствовать год ребёнок сейчас год удалить а...,274,34,6.852941,Жалобы
8,"Здравствуйте! Скажите, пожалуйста, может ли ми...",female,2021-02-24,здравствуйте скажите пожалуйста может ли минол...,здравствовать сказать пожалуйста ли минолексин...,122,15,7.200000,Жалобы
12,"Здравствуйте, Евгений Альбертович! Куда и с ка...",female,2021-02-14,здравствуйте евгений альбертович куда и с каки...,здравствовать евгений альбертович куда какой о...,157,20,6.900000,Запись на прием
...,...,...,...,...,...,...,...,...,...
6856,"здравствуйте! беспокою вас с сахалина, пореком...",male,2012-04-02,здравствуйте беспокою вас с сахалина порекомен...,здравствовать беспокою сахалин порекомендовать...,266,30,7.900000,Жалобы
6861,Здравствуйте! У меня муж ночью сильно храпит. ...,female,2012-03-25,здравствуйте у меня муж ночью сильно храпит чт...,здравствовать муж ночь сильно храпеть можно сд...,82,11,6.545455,Жалобы
6867,Здравствуйте. Подскажите когда и где в ближайш...,female,2012-02-22,здравствуйте подскажите когда и где в ближайше...,здравствовать подсказать где близкий время мож...,70,9,6.888889,Запись на прием
6871,Здравствуйте Евгений Альбертович! У меня к Вам...,female,2012-03-02,здравствуйте евгений альбертович у меня к вам ...,здравствовать евгений альбертович два вопрос к...,546,69,6.927536,Жалобы


# Определение ключевых слов
Для этого я воспользуюсь следующим алгоритмом: Преобразовать текст в граф и искать центральные узлы.

In [69]:
from sklearn.feature_extraction.text import TfidfVectorizer
kokee='\n'.join(info['lemmatized'].tolist())

In [68]:
meksna = kokee.split("\n")[1:-1]

In [33]:
from itertools import combinations
from collections import Counter
import np

In [34]:
def get_slov(text, top=10, window_size=5, random_p=0.1):
    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    ids = [word2id[word] for word in text]
    m = np.zeros((len(vocab), len(vocab)))
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        for j, k in combinations(window, 2):
            m[j][k] += 1
            m[k][j] += 1
    for i in range(m.shape[0]):
        s = np.sum(m[i])
        if not s:
            continue
        m[i] /= s
    
    c = Counter()
    n = np.random.choice(len(vocab))
    for i in range(500):
        go_random = np.random.choice([0, 1], p=[1-random_p, random_p])
        
        if go_random:
            n = np.random.choice(len(vocab))
        
        n = take_step(n, m)
        c.update([n])
    return [id2word[i] for i, count in c.most_common(top)]

def take_step(n, matrix):
    rang = len(matrix[n])
    if np.any(matrix[n]):
        next_n = np.random.choice(range(rang), p=matrix[n])
    else:
        next_n = np.random.choice(range(rang))
    return next_n

In [73]:
from nltk.tokenize import word_tokenize

In [74]:
for text in meksna:
    print(get_slov(word_tokenize(text)))

['послеоперационный', 'период', 'забота', 'год', 'лечение', 'оперативный', 'насколько', 'дышать', 'гилифанову', 'спать']
['ребёнок', 'сторона', 'год', 'слышать', 'тк', 'поворачивать', 'звук', 'удаление', 'выявить', 'время']
['привести', 'мг', 'сутки', 'дозировка', 'моноциклин', 'слух', 'нарушение', 'возникновение', 'шум', 'ухо']
['год', 'слух', 'повод', 'спасибо', 'снижение', 'последний', 'записаться', 'обследование', 'со', 'приём']
['ни', 'ближе', 'нос', 'глубоко', 'глоткебыли', 'лор', 'время', 'полип', 'плохо', 'отходить']
['боль', 'сустав', 'день', 'коленный', 'ждать', 'знать', 'каждый', 'документ', 'усиливаться', 'доктор']
['специалист', 'доктор', 'ухо', 'порекомендовать', 'часть', 'ли', 'конкретный', 'обращатьсянаблюдаться', 'регулярно', 'определённый']
['нос', 'полип', 'обоняние', 'говорить', 'ваш', 'потому', 'рот', 'увидеть', 'два', 'подарить']
['пока', 'аллергия', 'дом', 'лечим', 'большой', 'неделя', 'оперативный', 'думать', 'чтото', 'спасибо']
['норма', 'челюсть', 'подняться',

['арбидол', 'антибиотик', 'пропить', 'какойто', 'принимать', 'следовать', 'валерия', 'совместно', 'отец', 'день']
['д', 'принимать', 'вопрос', 'ли', 'можно', 'iu', 'аквадетрима', 'препарат', 'приводить', 'витамин']
['просить', 'иннокентьевна', 'доктор', 'васс', 'уважение', 'лечение', 'елена', 'после', 'больший', 'валерий']
['давление', 'утро', 'препарат', 'вечер', 'приём', 'пульс', 'год', 'принимать', 'после', 'два']
['ответ', 'благодарить', 'заранее', 'два', 'приём', 'день', 'головокружение', 'возможность', 'чтото', 'мечник']
['спасибо', 'огромный', 'приём', 'добрый', 'другом', 'воздух', 'раньше', 'появиться', 'хватать', 'ответ']
['неравномерный', 'артерия', 'гамитович', 'нконечностей', 'год', 'оба', 'отдел', 'цвет', 'утолщение', 'правый']
['сейчас', 'приём', 'йод', 'симптом', 'принимать', 'горло', 'токсичный', 'тиосульфат', 'хороший', 'щитовидки']
['пцр', 'также', 'ли', 'антитела', 'титр', 'через', 'трихоманиаза', 'раз', 'александр', 'сказать']
['отделение', 'много', 'клещ', 'инфицир

['английский', '``', 'язык', 'профессия', 'вопрос', 'ли', 'будущий', 'пожалуйста', 'ответить', 'тема']
['ухо', 'слух', 'сказать', 'стать', 'добрый', 'пройти', 'слово', 'года', 'приходиться', 'хотеть']
['простудный', 'синус', 'молчащего', 'каждый', 'заболевание', 'городской', 'можно', 'синдром', 'сейчас', 'делать']
['возможный', 'карьерный', 'рост', 'ли', 'день', 'город', 'какой', 'других', 'устроится', 'специальность']
['день', 'татьяна', 'заранее', 'штука', 'уважение', 'благодарить', 'др', 'таблетки', 'полиоксидония', 'lg']
['укол', 'делать', 'колоть', 'помогать', 'препарат', 'возраст', 'отлично', 'школьный', 'местный', 'работать']
['большой', 'увеличить', 'пч', 'какой', 'здравствовать', 'объёме', 'искривление', 'убрать', 'рассчитывать', 'сумиллиметру']
['сеанс', 'унитаз', 'первый', 'здравствовать', 'потом', 'уйти', 'прошёл', 'доктор', 'многое', 'химия']
['сеанс', 'пёс', 'первый', 'последний', 'преднизолон', 'андрокори', 'промежуток', 'неделя', 'сантиметрысла', 'туалет']
['возраст', '

['начало', 'глотание', 'конец', 'становиться', 'аллерголог', 'спасибо', 'простуде', 'ощущение', 'горло', 'иногда']
['цвет', 'здравствовать', 'реакция', 'красный', 'аллергический', 'записываться', 'стоять', 'приём', 'татуировка']
['день', 'герпес', 'вылезть', 'ежегодно', 'назад', 'место', 'бывать', 'добрый', 'положение', 'влиять']
['анализ', 'приём', 'сдача', 'кровь', 'окончание', 'капля', 'лекарство', 'расширить', 'статус', 'следующий']
['получить', 'мазь', 'помощь', 'там', 'вобщем', 'момент', 'обращение', 'хирург', 'нога', 'можно']
['делать', 'клиника', 'стафилокок', 'знать', 'специфика', 'здраствуйте', 'показать', 'ближе', 'лечение', 'анализ']
['делать', 'жить', 'обратитсяя', 'деревня', 'можно', 'месяц', 'проходить', 'здравствовать', 'боль', 'третий']
['пожалуйста', 'медикаментозный', 'хороший', 'аборт', 'сказать', 'владивосток', 'где', 'здравствовать', 'стоять', 'сколько']
['сбор', 'дальнейший', 'пуповинный', 'хранение', 'кровь', 'хотеться', 'шанс', 'опыт', 'год', 'предстоять']
['вл

['глаз', 'травма', 'роговица', 'отличный', 'видеть', 'близко', 'зрение', 'можноли', 'интересовать', 'боковой']
['здоровье', 'процветание', 'вселенский', 'спасибо']
['внутренний', 'путь', 'валерий', 'глаз', 'решение', 'дмитриевич', 'возле', 'убрать', 'ли', 'проходить']
['делать', 'попасть', 'операциюслышала', 'приморцам', 'сейчас', 'диабет', 'здравствовать', 'годакатаракта', 'можно', 'глазакак']
['год', 'катаракт', 'можно', 'глаукома', 'добрый', 'валерия', 'дмитриевич', 'бабушка', 'помочь', 'день']
['лицевой', 'защемление', 'птоз', 'верхний', 'нерв', 'игоревна', 'травма', 'родовой', 'здравствовать', 'юлия']
['юлия', 'игоревна', 'ухо', 'около', 'здравствовать', 'жировой', 'удобный', 'сколько', 'щека', 'шарик']
['год', 'ухом', 'правый', 'удариться', 'хрустеть', 'челюсть', 'куда', 'обратиться', 'упасть', 'можно']
['удаление', 'вечер', 'узнать', 'хотеться', 'добрый', 'ринит', 'палипозатрофический', 'хронический', 'бронхиальный', 'аспириновая']
['рукахчто', 'помогитепожалуйста', 'медик', 'пр

...

['невролог', 'приём', 'обратиться', 'подсказать', 'деньмне', 'боль', 'непрятные', 'ощущение', 'добрый', 'б']
['прописать', 'день', 'поставить', 'аевит', 'тело', 'после', 'добрый', 'эндомертиоз', 'помочь', 'поливитамины']
['длина', 'толшену', 'узнать', 'операциии', 'вопрос', 'хотеть', 'орган', 'стоимость', 'гепатит', 'юрий']
['здравствовать', 'сказать', 'половый', 'увеличение', 'член', 'операцыи', 'гепатомб', 'проводить']
['вернулисьтак', 'год', 'приступы', 'приступ', 'ли', 'продолжаться', 'выкидыш', 'случиться', 'очень', 'детство']
['сделать', 'необходимый', 'мрт', 'год', 'наблюдаться', 'какой', 'зрение', 'здравствовать', 'юлия', 'муж']
['день', 'ампул', 'ребёнок', 'через', 'понимать', 'итого', 'аспаркам', 'спать', 'где', 'очень']
['признак', 'час', 'инсульт', 'опухоль', 'хуже', 'проблема', 'отклонение', 'снимка', 'появиться', 'стать']
['сказать', 'полюс', 'удар', 'проживание', 'помочь', 'подскажете', 'поправить', 'время', 'деньга', 'находка']
['оперировать', 'диагноз', 'возьметесь', '

['удаление', 'киста', 'одновременный', 'исправление', 'носовой', 'операция', 'ли', 'гаймаровых', 'пазухах', 'перегородка']
['слух', 'нос', 'лор', 'неделя', 'рот', 'лечимсявсе', 'учиться', 'заложенного', 'начинаться', 'мучить']
['мезотимпанит', 'диагноз', 'двухсторонний', 'хронический', 'гнойный', 'слабость', 'нарушение', 'день', 'повысить', 'сонливость']
['владивосток', 'дома', 'можно', 'евгений', 'альбертовичкак', 'излечиться', 'храп', 'александр', 'одинс', 'уважение']
['називин', 'больница', 'неделя', 'назначить', 'потом', 'лежать', 'ребёнок', 'здравствовать', 'капать', 'физраствором']
['посоветовать', 'консультация', 'попасть', 'дело', 'ольга', 'алексеевич', 'бушуев', 'томчто']
['консультация', 'эстетический', 'ткань', 'хрящевой', 'добрый', 'нос', 'спасибо', 'попасть', 'заранее', 'куда']
['тип', 'экссудат', 'ребёнок', 'дело', 'ли', 'знать', 'средний', 'возраст', 'регистрироваться', 'отитсекреторная']
['лечить', 'часто', 'гайморит', 'полип', 'назад', 'совсем', 'спасибо', 'операция', 

['дефшейки', 'непоступподвнеподв', 'головка', 'сперматогенез', 'нормморфологии', 'очень', 'дефект', 'клетка', 'дефжизнеспособностьнд', 'сантиметрогу']
['горло', 'боль', 'исчезнуть', 'норма', 'красный', 'гранулы', 'температура', 'ли', 'возможный', 'после']
['максим', 'образ', 'предыдущий', 'дополнение', 'посоветуете', 'евгений', 'альбертович', 'киста', 'какой', 'необходимый']
['x', 'выполнять', 'правый', 'субтотально', 'сободные', 'командировка', 'посоветуйте', 'связь', 'левый', 'пожалуйста']
['проживать', 'петропавловскекамчатском', 'альбертович', 'евгений', 'здравствовать', 'консультация', 'александровна', 'уважение', 'прилететь', 'марианна']
['аденоиды', 'ли', 'назначить', 'сторонний', 'состоим', 'учёт', 'спортивный', 'степень', 'экссудативный', 'летдиагноз']
['месяц', 'меньерадо', 'препарат', 'бетагистин', 'евгений', 'мед', 'диагноз', 'посоветуете', 'пить', 'добавить']
['год', 'сад', 'здравствовать', 'сын', 'прошлый', 'альбертович', 'болеть', 'евгений', 'почти', 'пробовать']
['пожал

['нос', 'правый', 'посоветовать', 'болеть', 'киста', 'сантиметроркании', 'пазухино', 'левый', 'лобный', 'трудомголова']
['показать', 'поликлиника', 'порошок', 'чувствит', 'разводить', 'снова', 'госуд', 'нужный', 'saureus', 'рост']
['доктор', 'проблема', 'здравствовать', 'много', 'следующий', 'беспокоить', 'нос', 'сделать', 'горло', 'нарушить']
['попсть', 'консультация', 'рука', 'приём', 'правый', 'собираться', 'здравствовать', 'удалить', 'киста', 'гайморов']
['член', 'увеличить', 'рост', 'мало', 'волос', 'день', 'половый', 'добрый', 'год', 'почему']
['муж', 'данный', 'здравствовать', 'рн', 'сперматазоидов', 'спермотазоидову', 'ваш', 'юрий', 'александрович', 'отсутствие']
['никакой', 'камень', 'ли', 'беспокойство', 'симптом', 'дробление', 'возможный', 'муж', 'обнаружить', 'размер']
['сказать', 'полип', 'вопрос', 'здравствовать', 'верхнечелюстной', 'искривление', 'заключение', 'пазуха', 'мрт', 'начало']
['№', 'гнойный', 'антибиотик', 'хронический', 'опять', 'картина', 'назначение', 'отде

['подсказать', 'день', 'равный', 'альбертович', 'евгений', 'нужный', 'косточка', 'сантиметрогу', 'делать', 'лор']
['последний', 'лор', 'обратиться', 'боль', 'пить', 'мрт', 'невролог', 'размер', 'периодический', 'сантиметр']
['дочь', 'делать', 'владивосток', 'можно', 'голос', 'кокорин', 'проживать', 'виктория', 'ли', 'евгений']
['другое', 'чтолибо', 'заменить', 'пропить', 'возможность', 'ли', 'антибиотик', 'поставить', 'амоксиклав', 'диагноз']
['хронический', 'тонзилит', 'врач', 'горло', 'делать', 'каждый', 'евгений', 'вопрос', 'имиллиметрунологу', 'день']
['лазерный', 'связатьсязаписаться', 'васкак', 'можно', 'специалист', 'областив', 'какой', 'хабаровскеподскажитепожалуйстапо', 'телефон', 'никто']
['нос', 'усиливаться', 'закапать', 'переносица', 'капля', 'нажатиипробовала', 'пройти', 'день', 'насантиметрорк', 'после']
['западный', 'подсказать', 'схема', 'степень', 'возможный', 'лечение', 'перепробовать', 'метод', 'аденоиды', 'ребёнок']
['лоб', 'необходимость', 'глубь', 'глаз', 'район'